## Preprocess

In [128]:
import os
import pandas as pd
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# baseline config
class CFG:
    train_window_size = 90
    predict_size = 21
    epochs = 10
    learning_rate = 1e-3
    batch_size = 1024 * 3
    seed = 41

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG.seed) # Seed 고정

assert torch.cuda.is_available() == True, 'Not working GPU'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

### datasets
- train.csv : ID, 제품명, 대분류, 중분류, 소분류, 브랜드, 일자별 판매량
- sales.csv : ID, 일자별 판매액
- brand_keyword_cnt.csv : 브랜드별 검색 키워드량 (scaled)

In [129]:
train_df = pd.read_csv('data/train.csv')
keyword_df = pd.read_csv('data/brand_keyword_cnt.csv')
sales_df = pd.read_csv('data/sales.csv')

In [130]:
# check nan
f = lambda df : df.isna().sum().sum()
print(f'train_df = {f(train_df)}')
print(f'keyword_df = {f(keyword_df)}')
print(f'sales_df = {f(sales_df)}')

train_df = 0
keyword_df = 16065
sales_df = 0


In [131]:
# fill 0
keyword_df = keyword_df.fillna(0)
print(f(keyword_df))

0


In [132]:
# Drop columns
train_df = train_df.drop(['ID', '제품'], axis=1)
sales_df = sales_df.drop(['ID', '제품'], axis=1)

In [133]:
train_df.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
keyword_df.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.84131,0.91383,1.450530,2.422390,1.871190,1.581080,1.232950,1.174930,1.145920,...,0.319110,0.391640,0.377130,0.49318,0.072520,0.29010,0.31911,0.232080,0.333620,0.44966
1,B002-00002,12.64868,20.27850,15.332170,12.750210,13.562510,13.707570,11.937910,15.564250,14.084710,...,10.269790,11.966920,10.646930,10.41485,10.487380,9.48651,9.28343,10.429350,11.154620,11.38671
2,B002-00003,0.33362,0.43516,0.362630,0.174060,0.217580,0.464170,0.420650,0.290100,0.377130,...,0.536690,0.696250,0.449660,0.39164,1.029880,0.49318,0.91383,0.797790,1.015370,0.88482
3,B002-00005,1.07339,1.71163,2.016240,1.914700,1.987230,2.146790,1.682620,1.378000,1.421520,...,2.219320,2.509420,2.872060,2.37888,2.030750,1.53756,1.34899,1.261960,2.320850,2.30635
4,B002-00006,0.00000,0.00000,0.188558,0.246574,0.246574,0.246574,0.377139,0.087012,0.261084,...,0.072526,0.290103,0.087012,0.00000,0.130542,0.00000,0.00000,0.072526,0.217577,0.00000


In [135]:
sales_df.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,22400,67200,44800,0,0,44800,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### train label encoding

In [136]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
columns = train_df.columns[:4]
for col in columns:
    if col == '브랜드':
        keyword_df[col] = label_encoder.fit_transform(keyword_df[col])
        train_df[col] = label_encoder.transform(train_df[col])
        sales_df[col] = label_encoder.transform(sales_df[col])
    else:
        train_df[col] = label_encoder.fit_transform(train_df[col])
        sales_df[col] = label_encoder.transform(sales_df[col])

In [137]:
train_df.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,6,37,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,7,43,1,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,2,7,43,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,7,43,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
keyword_df.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,0.84131,0.91383,1.450530,2.422390,1.871190,1.581080,1.232950,1.174930,1.145920,...,0.319110,0.391640,0.377130,0.49318,0.072520,0.29010,0.31911,0.232080,0.333620,0.44966
1,1,12.64868,20.27850,15.332170,12.750210,13.562510,13.707570,11.937910,15.564250,14.084710,...,10.269790,11.966920,10.646930,10.41485,10.487380,9.48651,9.28343,10.429350,11.154620,11.38671
2,2,0.33362,0.43516,0.362630,0.174060,0.217580,0.464170,0.420650,0.290100,0.377130,...,0.536690,0.696250,0.449660,0.39164,1.029880,0.49318,0.91383,0.797790,1.015370,0.88482
3,3,1.07339,1.71163,2.016240,1.914700,1.987230,2.146790,1.682620,1.378000,1.421520,...,2.219320,2.509420,2.872060,2.37888,2.030750,1.53756,1.34899,1.261960,2.320850,2.30635
4,4,0.00000,0.00000,0.188558,0.246574,0.246574,0.246574,0.377139,0.087012,0.261084,...,0.072526,0.290103,0.087012,0.00000,0.130542,0.00000,0.00000,0.072526,0.217577,0.00000


In [139]:
sales_df.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,6,37,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,7,43,1,0,0,0,0,0,0,...,0,0,0,22400,67200,44800,0,0,44800,0
2,2,7,43,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,7,43,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### discount information
- 판매액을 판매량으로 나눠 개당 판매당시 가격을 계산
- 정보를 모르는 경우 0으로 저장

In [140]:
sales_col = train_df.columns[4:]
eps = 1e-8

discount_df = (sales_df[sales_col] / (train_df[sales_col] + eps)).fillna(0).round()

discount_df.head()

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,22400.0,22400.0,22400.0,0.0,0.0,22400.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
discount_df.min().min(), discount_df.max().max()

(0.0, 854118.0)

### scaling
- MinMaxScaler 이용

In [142]:
class MinMaxScaler:
    """ MinMaxScaler
    - Refence : https://dacon.io/competitions/official/236129/codeshare/8656?page=1&dtype=recent
    """
    def __init__(self):
        self.scale_min_dict = None
        self.scale_max_dict = None

    def fit_transform(self, df, sidx):
        cols = df.columns[sidx:]
        min_values = df[cols].min(axis=1)
        max_values = df[cols].max(axis=1)
        self.scale_min_dict = min_values.to_dict()
        self.scale_max_dict = max_values.to_dict()
        ranges = max_values - min_values
        ranges[ranges==0] = 1
        df[cols] = df[cols].subtract(min_values, axis=0).div(ranges, axis=0)
        return df

    def inverse_transform(self, pred):
        for idx in range(len(pred)):
            pred[idx, :] = pred[idx, :] * (self.scale_max_dict[idx] - self.scale_min_dict[idx]) + self.scale_min_dict[idx]
        return pred

In [143]:
scaler = MinMaxScaler()
scaled_train_df = scaler.fit_transform(train_df, 4)

In [144]:
scaled_train_df.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,6,37,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,2,7,43,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.0,0.0,0.222222,0.0
2,2,7,43,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,2,7,43,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0,0,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [145]:
# train information scaling
info_df = train_df[train_df.columns[:4]]

min_value = info_df.min(axis=0)
max_value = info_df.max(axis=0)

scaled_train_df[scaled_train_df.columns[:4]] = info_df.subtract(min_value, axis=1).div(max_value - min_value, axis=1)
scaled_train_df.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.25,0.6,0.711538,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,0.50,0.7,0.826923,0.000316,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.0,0.0,0.222222,0.0
2,0.50,0.7,0.826923,0.000316,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,0.50,0.7,0.826923,0.000316,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0.00,0.0,0.038462,0.000631,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [146]:
# keyword information scaling
info_df = keyword_df[['브랜드']]

min_value = info_df.min(axis=0)
max_value = info_df.max(axis=0)

keyword_df['브랜드'] = info_df.subtract(min_value, axis=1).div(max_value - min_value, axis=1)
keyword_df.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.000000,0.84131,0.91383,1.450530,2.422390,1.871190,1.581080,1.232950,1.174930,1.145920,...,0.319110,0.391640,0.377130,0.49318,0.072520,0.29010,0.31911,0.232080,0.333620,0.44966
1,0.000316,12.64868,20.27850,15.332170,12.750210,13.562510,13.707570,11.937910,15.564250,14.084710,...,10.269790,11.966920,10.646930,10.41485,10.487380,9.48651,9.28343,10.429350,11.154620,11.38671
2,0.000631,0.33362,0.43516,0.362630,0.174060,0.217580,0.464170,0.420650,0.290100,0.377130,...,0.536690,0.696250,0.449660,0.39164,1.029880,0.49318,0.91383,0.797790,1.015370,0.88482
3,0.000947,1.07339,1.71163,2.016240,1.914700,1.987230,2.146790,1.682620,1.378000,1.421520,...,2.219320,2.509420,2.872060,2.37888,2.030750,1.53756,1.34899,1.261960,2.320850,2.30635
4,0.001262,0.00000,0.00000,0.188558,0.246574,0.246574,0.246574,0.377139,0.087012,0.261084,...,0.072526,0.290103,0.087012,0.00000,0.130542,0.00000,0.00000,0.072526,0.217577,0.00000


In [147]:
# check keyword_df max value
df = keyword_df[keyword_df.columns[1:]]
df.min().min(), df.max().max()

(0.0, 13383.3178)

keyword_df도 MinMaxScaler 적용
- 범위가 0에서 최대 13383이기 때문에 모델에 바로 적용하기 어렵다고 생각했습니다.
- 이동평균은 고려하지 않았습니다.
    - 검색 키워드는 어떠한 이벤트에 의해 peak를 발생시킬 수 있고 판매량에 반영된다는 생각을 했습니다.
    - 따라서 검색량의 분포를 그대로 가져가면서 0과 1사이의 값으로 스케일링하는 것을 목표로 했습니다.

In [148]:
keyword_scaler = MinMaxScaler()
scaled_keyword_df = keyword_scaler.fit_transform(keyword_df, 1)

In [149]:
scaled_keyword_df.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.000000,0.214576,0.234817,0.384615,0.655870,0.502025,0.421053,0.323886,0.307693,0.299596,...,0.068825,0.089069,0.085019,0.117410,0.000000,0.060729,0.068825,0.044535,0.072875,0.105263
1,0.000316,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.158034,0.209827,0.169544,0.162461,0.164675,0.134130,0.127933,0.162904,0.185038,0.192120
2,0.000631,0.004359,0.006707,0.005030,0.000671,0.001677,0.007378,0.006372,0.003353,0.005365,...,0.009054,0.012743,0.007042,0.005701,0.020456,0.008048,0.017773,0.015091,0.020121,0.017103
3,0.000947,0.094303,0.180747,0.222004,0.208251,0.218075,0.239686,0.176818,0.135560,0.141454,...,0.249510,0.288801,0.337918,0.271121,0.223969,0.157171,0.131631,0.119843,0.263261,0.261297
4,0.001262,0.000000,0.000000,0.091544,0.119711,0.119711,0.119711,0.183100,0.042244,0.126755,...,0.035211,0.140844,0.042244,0.000000,0.063378,0.000000,0.000000,0.035211,0.105633,0.000000


In [156]:
scaled_keyword_df.min().min(), scaled_keyword_df.max().max()

(0.0, 1.0)

discount_df도 MinMaxScaler 사용
- 범위가 0부터 854118이므로 0부터 1사이의 값으로 스케일링이 필요하다고 생각했습니다.

In [151]:
discount_df.min().min(), discount_df.max().max()

(0.0, 854118.0)

In [158]:
discount_scaler = MinMaxScaler()
scaled_discount_df = discount_scaler.fit_transform(discount_df, 1)

In [159]:
scaled_keyword_df.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.000000,0.214576,0.234817,0.384615,0.655870,0.502025,0.421053,0.323886,0.307693,0.299596,...,0.068825,0.089069,0.085019,0.117410,0.000000,0.060729,0.068825,0.044535,0.072875,0.105263
1,0.000316,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.158034,0.209827,0.169544,0.162461,0.164675,0.134130,0.127933,0.162904,0.185038,0.192120
2,0.000631,0.004359,0.006707,0.005030,0.000671,0.001677,0.007378,0.006372,0.003353,0.005365,...,0.009054,0.012743,0.007042,0.005701,0.020456,0.008048,0.017773,0.015091,0.020121,0.017103
3,0.000947,0.094303,0.180747,0.222004,0.208251,0.218075,0.239686,0.176818,0.135560,0.141454,...,0.249510,0.288801,0.337918,0.271121,0.223969,0.157171,0.131631,0.119843,0.263261,0.261297
4,0.001262,0.000000,0.000000,0.091544,0.119711,0.119711,0.119711,0.183100,0.042244,0.126755,...,0.035211,0.140844,0.042244,0.000000,0.063378,0.000000,0.000000,0.035211,0.105633,0.000000


In [157]:
scaled_discount_df.min().min(), scaled_discount_df.max().max()

(0.0, 1.0)

In [162]:
scaled_train_df.to_csv('data/scaled_train.csv', index=False)
scaled_keyword_df.to_csv('data/scaled_keyword.csv', index=False)
scaled_discount_df.to_csv('data/scaled_discount.csv', index=False)